### **PART 1 用到preparation中的函数**

    例如：  
        * R = (ABCDEF)  
        * F = {A → B, B → CD, D → E, CE → F}  

In [1]:
R = ['A','B','C','D','E','F']

F = [[['A'],['B']],[['B'],['C','D']],[['D'],['E']],[['C','E'],['F']]]

**1.1 求属性闭包**  
1.单个属性  
2.所有属性  

In [2]:
def calculate_single_attribute_closure(original_alpha, F):
    """
    此函数用于计算单个属性的属性闭包
    ex:
    输入:['A'], F
    输出:['A','B','C','D','E','F'] 
    """
    alpha = original_alpha.copy()
    if alpha == []:
        return []
    result = alpha.copy()
    while True:
        for alpha_i, beta_i in F:
            #for α_i-->β_i, if α_i ⊂ result 
            if set(alpha_i).issubset(result):
                # result = result ∪ α_i
                result.extend([b for b in beta_i if b not in result])
        #if not changed after a loop, the calculation is finished 
        # 结果不再改变，说明属性闭包计算完成
        if result == alpha:
            break
        #if not finished, update alpha and continue calculations and comparisons 
        # 结果有所改变，继续计算和比较 
        alpha = result 
    return result

In [3]:
def calculate_attribute_closure_set( F ):
    """
    用来计算所有的属性闭包
    ex.
    输入:函数依赖集合F
    输出: 
    ['A']  -->  ['A', 'B', 'C', 'D', 'E', 'F']
    ['B']  -->  ['B', 'C', 'D', 'E', 'F']
    ['D']  -->  ['D', 'E']
    ['C', 'E']  -->  ['C', 'E', 'F']
    """
    #to store all attribute closures 用与存储所有的属性闭包
    attribute_closure_set = []
    for alpha, beta in F:
        result = calculate_single_attribute_closure(alpha, F)
        #to eliminate duplicate attribute closure 若属性闭包未重复，则添加
        if [alpha,sorted(result)] not in attribute_closure_set:
            attribute_closure_set.append([alpha,sorted(result)])
    return attribute_closure_set 

#calculate all attribute closures
attribute_closure_set = calculate_attribute_closure_set( F ) 

#print all attribute closures
for alpha, alpha_closure in attribute_closure_set:
    print(alpha,' --> ',alpha_closure)

['A']  -->  ['A', 'B', 'C', 'D', 'E', 'F']
['B']  -->  ['B', 'C', 'D', 'E', 'F']
['D']  -->  ['D', 'E']
['C', 'E']  -->  ['C', 'E', 'F']


**1.2 求候选码**

In [4]:
def get_subsets(s):
    """
    此函数用来计算集合的子集
    ex.
    输入:['A','B']
    输出:[[],['A'],['B'],['A','B']]
    #这个函数会使传入的s被修改, 需传入s.copy()
    """
    if not s:
        return [[]]
    x = s.pop()
    subsets = get_subsets(s)
    return subsets + [subset + [x] for subset in subsets]

def get_proper_subsets(s):
    """
    此函数用来计算集合的真子集
    ex.
    输入:['A','B']
    输出:[[],['A'],['B']]
    """
    s_copy = s.copy()
    temp = s.copy()
    subsets = get_subsets(temp)
    subsets.remove(s_copy)
    return subsets

def determine_candidate_key(alpha,F,R):
    """
    此函数用来判断属性集合是否为候选码
    ex.
    输入:['A'],F
    输出:True
    输入:['B']
    输出:False
    输入:['A','B']
    输出:False(此为超码而不是候选码)
    """
    #print("R",calculate_single_attribute_closure(alpha,F))
    if(set(R).issubset(calculate_single_attribute_closure(alpha,F))):

        proper_subsets_of_alpha = get_proper_subsets(alpha)
        sorted_proper_subsets_of_alpha = sorted(proper_subsets_of_alpha, key=len, reverse=True)
        is_candidate_key = True
        for item in sorted_proper_subsets_of_alpha:
            result = calculate_single_attribute_closure(item,F)
            if set(R).issubset(result):
                is_candidate_key = False
                break
        return is_candidate_key
    return False

def select_candidate_key(F):
    """
    此函数用来算出候选码
    ex.
    输出:['A']
    """
    candidate_key_set = []
    for alpha, beta in F:
        #print(alpha)
        temp = alpha.copy()
        if(determine_candidate_key(alpha,F,R)):
            candidate_key_set.append(temp)
    return candidate_key_set

candidate_key_set = select_candidate_key(F)

print(candidate_key_set)

[['A']]


**1.3 求Fc**

In [5]:
def function_dependency_union(F):
    """
    此函数返回合并后的函数依赖集合
    ex.
    F 中有 A-->B, B-->C, A-->C
    返回 A-->(B,C) , B-->C
    即 [[['A'], ['B', 'C']], [['B'], ['C']]]
    """
    union_F = []
    for alpha, beta in F:
        union_beta = beta.copy()
        for alpha_i, beta_i in F:
            if alpha == alpha_i:
                union_beta.extend([b for b in beta_i if b not in union_beta])
        if [alpha,sorted(union_beta)] not in union_F:
            union_F.append([alpha,sorted(union_beta)])
    return union_F

def eliminate_left_extraneous(union_F):
    """ 
    此函数用来剔除依赖关系集合F中左侧的extraneous attribute
    """
    for i, (alpha, beta) in enumerate(union_F):
        replace = [alpha, beta]
        for subset in get_proper_subsets(alpha):
            if subset:
                new_closure = calculate_single_attribute_closure(subset, union_F)
                if set(beta).issubset(new_closure) and (len(subset) < len(replace[0])):
                    replace = [subset, beta]
        union_F[i] = replace
    return union_F

def eliminate_right_extraneous(union_F):
    """ 
    此函数用来剔除依赖关系集合F中右侧的extraneous attribute
    """
    for i, (alpha, beta) in enumerate(union_F):
        replace = [alpha, beta]
        for subset in get_proper_subsets(beta):
            if subset:
                temp_union_F = union_F.copy()
                temp_union_F[i] = [alpha, subset]
                new_closure = calculate_single_attribute_closure(alpha, temp_union_F)
                if set(beta).issubset(new_closure) and (len(subset) < len(replace[1])):
                    replace = [alpha, subset]
        union_F[i] = replace
    return union_F

def compute_canonical_cover_set(F):
    """ 
    此函数用来获得依赖关系集合F的正则覆盖集
    ex.
    输入:F
    输出:F的正则覆盖集
    """
    F = function_dependency_union(F)
    F = eliminate_left_extraneous(F)
    F = function_dependency_union(F)
    F = eliminate_right_extraneous(F)
    return F

### **PART2 3NF判断**
    1. beta_belong_alpha(alpha,beta) return bool  
    2. alpha_is_superKey(alpha,F,R)    return bool  
    3. b_minus_a_contain_candidatKey(alpha,beta,F,R) return bool  

In [6]:
def beta_belong_alpha(alpha,beta):
    '''
    此函数用来判断beta是否属于alpha
    return bool
    '''
    return set(beta).issubset(set(alpha))
beta_belong_alpha(['A','B'],['A','B','C'])

False

In [7]:
def alpha_is_superKey(alpha,F,R):
    '''
    此函数用来判断alpha是否为超码
    return bool
    '''
    alpha_closure=calculate_single_attribute_closure(alpha,F)
    return set(R).issubset(set(alpha_closure))
alpha_is_superKey(['A'],F,R)

True

In [8]:
def b_minus_a_contain_candidatKey(alpha,beta,F,R):
    '''
    此函数用来判断beta-alpha是否包含于某个候选码中
    return bool
    '''
    a_set=set(alpha)
    b_set=set(beta)
    b_minus_a=list(b_set-a_set)
    candidatK=select_candidate_key(F)
    return b_minus_a in candidatK

alpha=['B']
beta=['A']
b_minus_a_contain_candidatKey(alpha,beta,F,R)


True

In [9]:
def test3NF(F,R):
    '''
    此函数用来测试表是否符合3NF
    return 破坏3NF的函数依赖关系
    当输出为[]时，代表表符合3NF
    '''
    break_F=[]
    for alpha,beta in F:
        if((not beta_belong_alpha(alpha,beta) and not alpha_is_superKey(alpha,F,R)and not b_minus_a_contain_candidatKey(alpha,beta,F,R))):
            break_F.append([alpha,beta])            
    return break_F
test3NF(F,R)

[[['B'], ['C', 'D']], [['D'], ['E']], [['C', 'E'], ['F']]]

### **PART3 做3NF分解**
    思路：  
        1.算Fc  
        2.每个依赖用一张表保存  
        3.求候选码  
        4.判断需不需要给候选码建表  
        5.删表:del_element(result)  return 删后的表
        6.输出  

In [10]:
def del_element(result):
    '''
    此函数用来删除有重复元素的表
    return 删后的表
    '''
    result.sort(key = lambda result:len(result))
    i=0
    while(i<(len(result))):
        temp=result.copy()
        result.pop(i)
        state=False
        for subs in result:
            if(set(temp[i]).issubset(set(subs))):
                state=True
        if(not state):
            result=temp
            i+=1 
    return result
a=[[1],[1,2],[1,2,3],[1,3,4],[1,4]]
del_element(a)


[[1, 2, 3], [1, 3, 4]]

In [11]:
def decomposition3NF(F,R):
    '''
    此函数用来根据3NF拆表
    return 拆后的表
    '''
    result=[]
    Fc=compute_canonical_cover_set(F)
    for alpha,beta in Fc:
        result.append(alpha+beta)
    candidatK=select_candidate_key(F)
    for canK in candidatK:
        result.append(canK)
    #print(result)
    result=del_element(result)
    return result

F=[[[1,2],[3,4]],[[2],[3]],[[1,3],[2]]]
R=[1,2,3,4]
print("F",F)
print("R",R)
print(decomposition3NF(F,R))

F [[[1, 2], [3, 4]], [[2], [3]], [[1, 3], [2]]]
R [1, 2, 3, 4]
[[1, 2, 4], [1, 3, 2]]


### **PART4 汇总**
先test再拆

In [12]:
def threeNF(F,R):
    '''
    此函数整合3NF的函数，先检验，再生成符合3NF的表
    输出是否破坏3NF，如果破坏，哪些函数依赖破坏3NF。
    return 符合3NF的表
    '''
    breakList=test3NF(F,R)
    if(breakList!=[]):
        print("存在函数依赖破坏3NF。")
        print("破坏3NF的函数依赖：",breakList)
        return decomposition3NF(F,R)
    else:
        print("该表符合3NF。")
        return R

threeNF(F,R)       

存在函数依赖破坏3NF。
破坏3NF的函数依赖： [[[2], [3]]]


[[1, 2, 4], [1, 3, 2]]